In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv008569/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()
spark

In [26]:
trendytech_base_rdd=spark.sparkContext.textFile("/public/trendytech/reviews/trendytech-student-reviews.csv")
trendyBoring_rdd=spark.sparkContext.textFile("/user/itv008569/boringwords.txt")
trendyBoring_rdd_boradcast=spark.sparkContext.broadcast(trendyBoring_rdd.collect()) # Broadcasted rdd
trendytech_base_flat=trendytech_base_rdd.flatMap(lambda x: x.split(" ")).map(lambda x: x.lower()).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
trendytech_base_filter=trendytech_base_flat.filter(lambda x: x[0] not in trendyBoring_rdd_boradcast.value).reduceByKey(lambda x,y: x+y).sortBy(lambda x:x[1], ascending=False)
#Filter using broadcasted rdd in lambda function
trendytech_base_filter.take(15)
#trendytech_base_flat.take(4)

[('data', 201),
 ('sumit', 109),
 ('trendytech', 67),
 ('', 64),
 ('data.', 34),
 ('course.', 33),
 ("sir's", 23),
 ('trendy', 14),
 ('course,', 13),
 ("master's", 13),
 ('domain.', 12),
 ("trendytech's", 12),
 ('sir.', 11),
 ('program.', 9),
 ('concepts.', 9)]